# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 744 ms, sys: 272 ms, total: 1.02 s
Wall time: 569 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.48 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_140_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh2/periodic_140_6561"
random_state = 782


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918540 entries, 0 to 918539
Columns: 148 entries, id to feat139
dtypes: float64(143), int32(3), object(2)
memory usage: 1.0+ GB
CPU times: user 12.1 s, sys: 834 ms, total: 12.9 s
Wall time: 15.5 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130839,-0.300829,...,-0.080270,0.115548,-0.174799,-0.244861,0.319462,0.059061,-0.000568,0.244879,-0.300829,-0.130839
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.130839,-0.300829,...,0.080270,0.115548,0.174799,-0.244861,-0.319462,0.059061,0.000568,0.244879,0.300829,-0.130839
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.136032,0.040156,...,0.174723,0.086763,-0.070199,0.105493,-0.137161,-0.181862,0.180933,0.052357,-0.040156,0.136032
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.016772,0.038115,...,0.011447,-0.002690,0.015595,0.024714,-0.039364,-0.009687,0.001017,-0.028996,0.038115,0.016772
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.096087,0.024817,...,0.105770,0.025590,-0.024172,0.086413,-0.100604,-0.112297,0.113588,0.022113,-0.024817,0.096087


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  140
n_hamiltonians:  6561
n_ts:  2
CPU times: user 154 µs, sys: 13 µs, total: 167 µs
Wall time: 112 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.8414875781130925
% val:  0.0
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  5521
number of val hamiltonians:  0
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [01:13<2:00:31, 73.05s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [02:22<1:57:36, 72.00s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [1:55:41<00:00, 68.70s/it]

running experiments: 100%|██████████| 100/100 [1:55:41<00:00, 69.42s/it]

CPU times: user 13h 1min 42s, sys: 16.5 s, total: 13h 1min 58s
Wall time: 1h 55min 41s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130839,-0.300829,...,-0.080270,0.115548,-0.174799,-0.244861,0.319462,0.059061,-0.000568,0.244879,-0.300829,-0.130839
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.130839,-0.300829,...,0.080270,0.115548,0.174799,-0.244861,-0.319462,0.059061,0.000568,0.244879,0.300829,-0.130839
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.136032,0.040156,...,0.174723,0.086763,-0.070199,0.105493,-0.137161,-0.181862,0.180933,0.052357,-0.040156,0.136032
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,0.016772,0.038115,...,0.011447,-0.002690,0.015595,0.024714,-0.039364,-0.009687,0.001017,-0.028996,0.038115,0.016772
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.096087,0.024817,...,0.105770,0.025590,-0.024172,0.086413,-0.100604,-0.112297,0.113588,0.022113,-0.024817,0.096087
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.037409,0.017407,...,0.064699,0.087166,-0.069474,-0.006214,-0.025629,-0.076251,0.078393,0.045301,-0.017407,0.037409
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,-0.001081,0.008635,...,-0.052698,-0.054268,0.035200,-0.005245,0.001839,0.033479,-0.012524,-0.019661,0.008635,-0.001081
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.037034,-0.082887,...,-0.029095,-0.011589,-0.023527,-0.045985,0.084698,0.025950,-0.004245,0.059527,-0.082887,-0.037034
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.105296,0.019814,...,0.076874,-0.034699,0.019704,0.110168,-0.114864,-0.090059,0.096749,-0.003643,-0.019814,0.105296
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.045950,0.023837,...,0.071514,0.118769,-0.103239,-0.027565,-0.023510,-0.100008,0.109383,0.069605,-0.023837,0.045950


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 103 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
727408,5195,H_1.2_-1.45_1.00111.dat,1.20,-1.45,1.001105,1,1,train,0.058886,0.060284,...,0.040755,0.051891,-0.133445,-0.033895,0.089522,0.118031,-0.130913,-0.101846,0.060284,0.058886
3811,27,H_-2_-0.65_-1.00345.dat,-2.00,-0.65,-1.003455,-1,-1,val,-0.078486,-0.009506,...,0.057838,-0.123184,-0.116268,0.022511,-0.020428,0.116283,0.123194,-0.054552,-0.009506,-0.078486
31929,228,H_-1.9_1.3_-0.000101676.dat,-1.90,1.30,-0.000102,0,0,val,0.082804,0.340767,...,0.057160,0.118630,-0.163981,-0.192448,0.239618,0.155784,-0.206599,-0.237666,0.340767,0.082804
293667,2097,H_-0.75_1.6_0.000669219.dat,-0.75,1.60,0.000669,0,0,val,-0.191875,-0.134074,...,0.146765,0.083857,0.258673,-0.034870,0.330151,-0.248179,0.062484,-0.289804,-0.134074,-0.191875
63322,452,H_-1.75_0.35_-0.00105877.dat,-1.75,0.35,-0.001059,0,0,train,0.044758,0.040843,...,0.159310,-0.060389,0.004627,-0.150502,-0.135611,0.001147,-0.041139,0.087540,0.040843,0.044758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518267,3701,H_0.25_0.8_2.00168.dat,0.25,0.80,2.001676,2,2,val,-0.040175,-0.028030,...,0.113608,-0.130554,-0.126990,0.125739,0.080515,-0.086880,-0.006559,0.070371,-0.028030,-0.040175
204765,1462,H_-1.1_-1.8_2.00883.dat,-1.10,-1.80,2.008833,2,2,val,0.193050,-0.132424,...,0.059429,0.091382,-0.014499,-0.135065,-0.024986,0.092015,-0.002258,-0.092365,-0.132424,0.193050
722022,5157,H_1.15_0.7_-1.00374.dat,1.15,0.70,-1.003736,-1,-1,train,-0.073408,-0.055687,...,-0.103264,-0.112296,0.000386,-0.037866,0.090116,0.073582,0.109825,0.134547,0.055687,0.073408
901766,6441,H_1.95_0.1_-1.00067.dat,1.95,0.10,-1.000674,-1,-1,val,-0.114266,-0.107872,...,0.114809,0.109745,0.073856,0.058848,0.001481,-0.010615,-0.063284,-0.076821,-0.107872,-0.114266


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.4208199969516842
% val:  0.42066758116140834
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  2761
number of val hamiltonians:  2760
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [2588, 2007, 5036, 544, 6048, 4409, 3917, 4090, 4002, 3326, 1576, 2665, 4663, 1517, 4092, 5155, 6360, 688, 4205, 2989, 6295, 283, 3570, 3741, 4544, 5934, 3068, 4309, 3166, 2667, 4601, 1198, 50, 2854, 792, 1124, 4290, 921, 351, 4789, 1597, 5063, 2768, 5925, 5565, 1403, 276, 2894, 6091, 791, 5731, 215, 4944, 6250, 1990, 5680, 5139, 2308, 3828, 2972, 631, 5184, 1515, 1442, 1610, 4739, 4727, 6494, 3084, 5688, 2518, 5043, 1296, 3667, 6401, 2116, 1181, 3447, 1290, 3041, 4424, 4136, 3216, 3758, 2748, 605, 5276, 4381, 5631, 4649, 2831, 5625, 4003, 4372, 5592, 2573, 6179, 2513, 4915, 2970, 5588, 2590, 3179, 1341, 6243, 4884, 5549, 3931, 394, 1944, 103, 4009, 34, 3745, 5922, 230, 590, 1775, 994, 3454, 1743, 2675, 3535, 3396, 5208,

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 49 µs, sys: 0 ns, total: 49 µs
Wall time: 51.3 µs


,t1,t2,type_of,-1,0,1,2,phase,pred_phase
id,,,,,,,,,
0,-2.0,-2.00,test,0.278571,0.314286,0.300000,0.107143,999,0
1,-2.0,-1.95,test,0.285714,0.328571,0.321429,0.064286,-1,0
2,-2.0,-1.90,test,0.257143,0.328571,0.321429,0.092857,-1,0
3,-2.0,-1.85,test,0.321429,0.385714,0.242857,0.050000,-1,0
4,-2.0,-1.80,test,0.342857,0.314286,0.264286,0.078571,-1,-1
...,...,...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.500000,0.200000,0.064286,0.235714,-1,-1
6557,2.0,1.85,test,0.442857,0.214286,0.057143,0.285714,-1,-1
6558,2.0,1.90,test,0.364286,0.278571,0.071429,0.285714,-1,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  918540


,id,phase,pred_phase,type_of
0,0,999,-1,test
1,0,999,-1,test
2,0,999,1,test
3,0,999,-1,test
4,0,999,1,test
...,...,...,...,...
918535,6560,999,1,test
918536,6560,999,1,test
918537,6560,999,-1,test
918538,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9997076628550732,
 'eigenvector_val': 0.9660326086956522,
 'eigenvector_test': 0.648586396141693,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 0.9967391304347826,
 'hamiltonian_test': 0.8928987194412107}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}